### Домашнее задание к лекции «Spark SQL»
##### Оглавление
[Задание 1.](#task1)   
[Задание 2.](#task2)  
[Задание 3.](#task3) 

<h5 style="color:Red">Задание 1.</h5><a id='task1'></a> - Выберите 15 стран с наибольшим процентом переболевших на 31 марта (в выходящем датасете необходимы колонки: iso_code, страна, процент переболевших)

In [117]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import * 

In [2]:
#создаем переменную с сессией 
spark = SparkSession.builder.appName("spark_sql").getOrCreate()

In [57]:
#читаем датафрейм
df = spark.read.option('header', True).option('inferSchema', True).format('csv').load('owid-covid-data.csv', sep = ',')

In [8]:
#проверяем, вроде получилось
df.show(5)

+--------+---------+-----------+----------+-----------+---------+------------------+------------+----------+-------------------+-----------------------+---------------------+------------------------------+------------------------+----------------------+-------------------------------+-----------------+------------+------------------------+-------------+-------------------------+---------------------+---------------------------------+----------------------+----------------------------------+---------+-----------+------------------------+----------------------+------------------+-------------------------------+-------------+--------------+-----------+------------------+-----------------+-----------------------+----------------+-------------------------+------------------------------+-----------------------------+-----------------------------------+-------------------------------------+----------------+----------+------------------+----------+-------------+-------------+--------------+---

In [96]:
""" я не смог найти колонку с количеством переболевших, 
поэтому решил взять рассмотреть 15 стран с наибольшим количеством смертей """
df_filtered = df.select('iso_code', 'location', 'total_deaths', 'date').filter(df['date'] == '2021-03-01')


In [97]:
#группируем отфильтрованный фрейм
df_groupped = df_filtered.groupBy('iso_code', 'location', 'total_deaths').max('total_deaths').sort(desc('total_deaths'))
df_groupped.show(15)


+--------+--------------+------------+-----------------+
|iso_code|      location|total_deaths|max(total_deaths)|
+--------+--------------+------------+-----------------+
|OWID_WRL|         World|   2539947.0|        2539947.0|
|OWID_EUR|        Europe|    817560.0|         817560.0|
|OWID_NAM| North America|    750075.0|         750075.0|
|OWID_EUN|European Union|    547909.0|         547909.0|
|     USA| United States|    515796.0|         515796.0|
|OWID_SAM| South America|    467842.0|         467842.0|
|OWID_ASI|          Asia|    399483.0|         399483.0|
|     BRA|        Brazil|    255720.0|         255720.0|
|     MEX|        Mexico|    186152.0|         186152.0|
|     IND|         India|    157248.0|         157248.0|
|     GBR|United Kingdom|    123187.0|         123187.0|
|OWID_AFR|        Africa|    104021.0|         104021.0|
|     ITA|         Italy|     97945.0|          97945.0|
|     FRA|        France|     86955.0|          86955.0|
|     RUS|        Russia|     8

<h5 style="color:Red">Задание 2.</h5><a id='task2'></a> -Top 10 стран с максимальным зафиксированным кол-вом новых случаев за последнюю неделю марта 2021 в отсортированном порядке по убыванию
(в выходящем датасете необходимы колонки: число, страна, кол-во новых случаев)

In [99]:
df_1 = spark.read.option('header', True).option('inferSchema', True).format('csv').load('owid-covid-data.csv', sep = ',')

In [124]:
df_cast = df_1.withColumn('date', col('date').cast('date'))

In [159]:
filtered_df_1 = df_cast.filter(
    col('date').between('2021-03-25', '2021-03-31')
).select('total_cases', 'location', 'new_cases').groupBy('location').count().alias('count').orderBy('count', ascending = False).show()

+-----------------+-----+
|         location|count|
+-----------------+-----+
|             Chad|    7|
|         Anguilla|    7|
|          Eritrea|    7|
|         Djibouti|    7|
|          Comoros|    7|
|      Afghanistan|    7|
|         Cambodia|    7|
|           Africa|    7|
|         Dominica|    7|
|          Algeria|    7|
|Equatorial Guinea|    7|
|        Argentina|    7|
|           Angola|    7|
|          Belgium|    7|
|          Ecuador|    7|
|          Albania|    7|
|            Benin|    7|
|          Bahamas|    7|
|            China|    7|
|          Belarus|    7|
+-----------------+-----+
only showing top 20 rows

